<h1>Statistics, graph measures and SVM classification</h1>

This notebook includes the main analysis of our data, after the preprocessing phase, and we will base our discussion on the results obtained here. The analysis phase can be summarized this way: first we derive a number of graph measures, both global and nodal, from our data, and we test whether there are statistically significant differences between groups regarding those graph measures. Only those measures that will deliver a statistically significant difference between groups will be used for the classification tasks.  We will be focusing mostly on testing differences between patients and controls, but we also check for possible differences between disease phenotype groups (RRMS vs. PPMS/SPMS). Because of the high amount of comparisons in each test, rather than implementing a Bonferroni correction for each test we have decided to use p<0.001 as a threshold to determine whether a graph measure is subsequently used for the classification task.

Next, we will execute several classification tasks, using Support Vector Machine (SVM). There will be three types of classification:

-Using our connectivity data (structural, functional and weighted combined)

-Using graph measures (global and nodal)

-Using brain volumes

We will also do here some further processing of the stregth graph measures that we will use for our visualizations.  


---------------

Index for the notebook:

- [1. Data loading](#a1)
- [2. Graph measures first try](#a2)
- [3. Functions for graph measures](#a3)
- [4. Graph measures](#a4)  
  * [4.1 Strength](#a4.1)  
  * [4.2 Node Degree](#a4.2)  
  * [4.3 Degree Centrality](#a4.3)  
  * [4.4 Closeness centrality](#a4.4)  
  * [4.5 Clustering coefficient](#a4.5)  
  * [4.6 Local Efficiency](#a4.6)  
  * [4.7 Global efficiency](#a4.7)  
  * [4.8 Betweenness Centrality](#a4.8)  
  * [4.9 Eigenvector Centrality](#a4.9)  
  * [4.10 Assortativity](#a4.10)  
  * [4.11 Rich Club](#a4.11)  
    + [4.11.1 Results](#a4.11.1)  
- [5. Classification](#a5)  
  * [5.1 Functions for FA classification](#a5.1)  
  * [5.2 Connectivity based classification](#a5.2)    
    + [5.2.1 Results](#a5.2.1)  
  * [5.3 Functions for graph based classification](#a5.3)  
  * [5.4 Classification using global graph measures](#a5.4)  
  * [5.5 Classification using vectors of diverse nodal graph measures](#a5.5)  
    + [5.5.1 Results](#a5.5.1)  
  * [5.6 Classification using brain volumes](#a5.6)  
    + [5.6.1 All brain volumes](#a5.6.1)  
    + [5.6.2 Thalamic volumes](#a5.6.2)  
    + [5.6.3 Five most different volumes](#a5.6.3)  
    + [5.6.4 Results](#a5.6.4)  
  * [5.7 Summary of classification results](#a5.7)  
- [6. Comparison of strength for connectogram visualization](#a6)
- [7. Preliminary conclusions](#a7)

In [1]:
import numpy as np
import pandas as pd
import glob

import networkx as nx

from scipy import stats

import os.path

from statistics import mean, median

<a id='a1'></a>
<h1>Data Loading</h1>

In [2]:
# we load the preprocessed data and we separate patients from controls

path_st = 'structural_ready'
path_func = 'functional_ready'

csv_files_st = [file for file in sorted(os.listdir(path_st), key=lambda x: x.lower())]
csv_files_func = [file for file in sorted(os.listdir(path_func), key=lambda x: x.lower())]

st_matrices = [pd.read_csv(os.path.join(path_st, file), header=None) for file in csv_files_st]
func_matrices = [pd.read_csv(os.path.join(path_func, file), header=None) for file in csv_files_func]

demographics_df = pd.read_csv('clinic.csv')
labels = list(demographics_df['controls_ms'])
ms_type = list(demographics_df['mstype'])

ms_st = [st_matrices[i] for i, value in enumerate(labels) if value == 1]
hv_st = [st_matrices[i] for i, value in enumerate(labels) if value == 0]

ms_func = [func_matrices[i] for i, value in enumerate(labels) if value == 1]
hv_func = [func_matrices[i] for i, value in enumerate(labels) if value == 0]



In [3]:
# We load the brain volume data

volumes_patients = pd.read_csv('volum_nodes_patients.csv') 
volumes_controls = pd.read_csv('volum_nodes_controls.csv')

In [4]:
volumes_patients

,0,1,2,3,4,5,6,7,8,9,...,67,68,69,70,71,72,73,74,75,76
0,002MSVIS,3672,7928,5064,3864,11512,14088,13424,2872,13240,...,13696,12888,2872,12704,31088,11536,16560,8784,848,7600
1,003MSVIS,4040,8544,4144,3208,12872,16688,13288,2736,14840,...,16176,11592,2808,13864,34160,14384,19168,13232,1312,7904
2,004MSVIS,5224,7656,4136,3192,11712,15728,14720,3960,13368,...,13696,11760,3304,14024,29896,12328,18824,12240,1304,7856
3,005MSVIS,4664,9064,4848,3520,13512,17696,14712,3960,13800,...,16216,13928,3000,14776,33904,12744,21024,11904,1576,10240
4,010MSVIS,5520,9056,4560,3560,12856,17048,14264,3848,13720,...,14696,12544,3656,14440,35600,12824,21200,13976,1608,9096
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,FIS_125,4456,8024,4408,3248,11744,16208,12800,3160,15304,...,13632,14624,3280,15440,34304,13008,18112,13392,1320,8296
143,FIS_126,4240,7904,4784,3080,12168,16088,14504,3336,13016,...,11784,12224,2432,12200,30408,11592,16376,11464,1288,8496
144,FIS_127,5296,9336,5064,3040,14488,18008,14432,4352,16080,...,16016,13288,3824,14944,39072,13288,20936,14240,1568,9312
145,FIS_128,4464,8048,4768,3216,13568,17120,14072,3608,17840,...,15312,13368,3856,13360,38984,14296,20208,12200,1752,8568


In [5]:
# We convert volume matrices to lists

volumes_ms = volumes_patients.values[:,1:].tolist()
volumes_hv = volumes_controls.values[:,1:].tolist()
volumes = volumes_ms + volumes_hv

In [6]:
# We show the patients vs. hv labels 
print(labels)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]


In [7]:
# Here are the phenotype labels
print(ms_type)

[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, -1, -1, -1, -1, -1, -1, -1]


In [8]:
# We filter out hv controls from the phenotype labels
ms_type_ms = [i for i in ms_type if i >= 0]
print(ms_type_ms)

[1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 1, 1, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [9]:
# We create 2 groups out of the 3 phenotypes: rrms vs. ppms/spms and we adjust the  lables accordingly 
ms_rr = [ms_st[i] for i, value in enumerate(ms_type_ms) if value == 0]
ms_psp = [ms_st[i] for i, value in enumerate(ms_type_ms) if value != 0]

ms_rr_func = [ms_func[i] for i, value in enumerate(ms_type_ms) if value == 0]
ms_psp_func = [ms_func[i] for i, value in enumerate(ms_type_ms) if value != 0]

ms_type_labels = [1 if i != 0 else i for i in ms_type_ms]
print(ms_type_labels)

[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [10]:
print(len(ms_st))
print(len(hv_st))

147
18


In [11]:
# We show a dictionary with the names of the 76 nodes

nodes_csv = pd.read_csv('nodes.csv', header=None)
nodes = nodes_csv[1].tolist()
nodes_dict = {i+1: j for i, j in enumerate(nodes)}
print(nodes_dict)

{1: 'ctx-lh-caudalanteriorcingulate', 2: 'ctx-lh-caudalmiddlefrontal', 3: 'ctx-lh-cuneus', 4: 'ctx-lh-entorhinal', 5: 'ctx-lh-fusiform', 6: 'ctx-lh-inferiorparietal', 7: 'ctx-lh-inferiortemporal', 8: 'ctx-lh-isthmuscingulate', 9: 'ctx-lh-lateraloccipital', 10: 'ctx-lh-lateralorbitofrontal', 11: 'ctx-lh-lingual', 12: 'ctx-lh-medialorbitofrontal', 13: 'ctx-lh-middletemporal', 14: 'ctx-lh-parahippocampal', 15: 'ctx-lh-paracentral', 16: 'ctx-lh-parsopercularis', 17: 'ctx-lh-parsorbitalis', 18: 'ctx-lh-parstriangularis', 19: 'ctx-lh-pericalcarine', 20: 'ctx-lh-postcentral', 21: 'ctx-lh-posteriorcingulate', 22: 'ctx-lh-precentral', 23: 'ctx-lh-precuneus', 24: 'ctx-lh-rostralanteriorcingulate', 25: 'ctx-lh-rostralmiddlefrontal', 26: 'ctx-lh-superiorfrontal', 27: 'ctx-lh-superiorparietal', 28: 'ctx-lh-superiortemporal', 29: 'ctx-lh-supramarginal', 30: 'ctx-lh-transversetemporal', 31: 'ctx-lh-insula', 32: 'Left-Thalamus-Proper', 33: 'Left-Caudate', 34: 'Left-Putamen', 35: 'Left-Pallidum', 36: '

In [12]:
# We create a mix of structural and functional data through a simple weighted sum, giving more 
# importance to structural connections

alpha = 0.75  # Weight for structural matrices
beta = 0.25   # Weight for functional matrices

all_combined = [alpha * structural + beta * functional for structural, functional in zip(st_matrices, func_matrices)]

ms_combined = [alpha * structural + beta * functional for structural, functional in zip(ms_st, ms_func)]
hv_combined = [alpha * structural + beta * functional for structural, functional in zip(hv_st, hv_func)]


<a id='a2'></a>
<h1>Graph measures (first try)</h1>

For archiving purposes, we include here our first statistical test based on the stregth measures of structural  matrices.

In [13]:
# We calculate first the strength measure of all individual graphs
# and the we average for the groups before implementing a test of independence between groups

strength_ms_st_list = []
strength_hv_st_list = []

for m in ms_st:
    G = nx.from_pandas_adjacency(m)
    strength = G.degree(weight='weight')
    strengths = {node: val for (node, val) in strength}
    strength_ms_st_list.append(strengths)

for m in hv_st:
    G = nx.from_pandas_adjacency(m)
    strength = G.degree(weight='weight')
    strengths = {node: val for (node, val) in strength}
    strength_hv_st_list.append(strengths)
    

global_strength_ms_st = [np.mean(list(s.values())) for s in strength_ms_st_list]
global_strength_hv_st = [np.mean(list(s.values())) for s in strength_hv_st_list]

In [14]:
# we obtain a significant difference p<0.001

t_stat, p_value = stats.ttest_ind(global_strength_ms_st, global_strength_hv_st)
u_stat, p_value_u = stats.mannwhitneyu(global_strength_ms_st, global_strength_hv_st)

if p_value < 0.001:
    print(f"t-statistic: {t_stat}, p < 0.001")
else:
    print(f"t-statistic: {t_stat}, p = {p_value:.3f}")

if p_value_u < 0.001:
    print(f"mann-whitney-statistic: {u_stat}, p < 0.001")
else:
    print(f"mann-whitney-statistic: {u_stat}, p = {p_value:.3f}")

t-statistic: -4.39126274327306, p < 0.001
mann-whitney-statistic: 393.0, p < 0.001


<a id='a3'></a>
<h1>Functions for graph measures</h1>

After this first try, we define several functions to obtain and test independence between groups using different graph measures, both global and nodal. The functions include normality tests to decide in each instance whether a t-test or a Mann-Whitney test should be performed.

In [15]:
# Graph measures calculation:

def calculate_graph_measures(data, measure='degree', weight='weight', network='global', layer='single'):
    measures_list = []
    
    for m in data:
        if layer == 'single':
            G = nx.from_pandas_adjacency(m)
        elif layer == 'multilayer':
            G = nx.from_numpy_array(m)
        if measure == 'degree':
            measure_result = G.degree(weight=weight)
            measures = {node: val for (node, val) in measure_result}
        elif measure == 'node_degree':
            measures = sum(dict(G.degree()).values())
        elif measure == 'degree_centrality':
            if network == 'global':
                measures = list(nx.degree_centrality(G).values())   
            else:
                measures = nx.degree_centrality(G)
        elif measure == 'closeness_centrality':
            if network == 'global':
                measures = list(nx.closeness_centrality(G).values())
            else:
                measures = nx.closeness_centrality(G)
        elif measure == 'average_clustering':
            measures = nx.average_clustering(G)
        elif measure == 'clustering':
            measures = nx.clustering(G)
        elif measure == 'local_efficiency':
            measures = nx.local_efficiency(G)
        elif measure == 'global_efficiency':
            measures = nx.global_efficiency(G)
        elif measure == 'betweenness_centrality':
            if network == 'global':
                measures = list(nx.betweenness_centrality(G).values())   
            else:
                measures = nx.betweenness_centrality(G)
        elif measure == 'eigenvector_centrality':
            if network == 'global':
                measures = list(nx.eigenvector_centrality(G).values())   
            else:
                measures = nx.eigenvector_centrality(G)
        elif measure == 'assortativity':
            measures = nx.degree_assortativity_coefficient(G)
        elif measure == 'rich_club':
            measures = nx.rich_club_coefficient(G, normalized=False, seed=42)
        else:
            raise ValueError(f"Unsupported graph measure: {measure}")
        
        measures_list.append(measures)
    return measures_list

In [16]:
# Independence test for global measures:

def stats_test(ms=ms_st, hv=hv_st, measure='degree', weight='weight', centrality='mean', network='global', 
               layer='single'):
    ms_list = calculate_graph_measures(ms, measure=measure, weight=weight, layer=layer)
    hv_list = calculate_graph_measures(hv, measure=measure, weight=weight, layer=layer)

    if measure == 'degree' or network != 'global':
        if centrality == 'mean':
            global_ms = [np.mean(list(s.values())) for s in ms_list]
            global_hv = [np.mean(list(s.values())) for s in hv_list]
        elif centrality ==  'median':
            global_ms = [np.median(list(s.values())) for s in ms_list]
            global_hv = [np.median(list(s.values())) for s in hv_list]
        else:
            raise ValueError(f"Unsupported centrality measure: {centrality}")
    elif (measure == 'node_degree' or
          measure == 'average_clustering' or
          measure == 'local_efficiency' or
          measure == 'global_efficiency' or
          measure == 'assortativity' or
          measure == 'clustering_coefficient'):
        global_ms = [s for s in ms_list]
        global_hv = [s for s in hv_list]
    else:
        if centrality == 'mean':
            global_ms = [mean(s) for s in ms_list]
            global_hv = [mean(s) for s in hv_list]
        elif centrality ==  'median':
            global_ms = [median(s) for s in ms_list]
            global_hv = [median(s) for s in hv_list]
        else:
            raise ValueError(f"Unsupported centrality measure: {centrality}")
   
    _, p_value_shapiro_ms = stats.shapiro(global_ms)
    _, p_value_shapiro_hv = stats.shapiro(global_hv)

    if p_value_shapiro_ms > 0.05 and p_value_shapiro_hv > 0.05:
        test = 'ttest'
        stat, p_value = stats.ttest_ind(global_ms, global_hv)
    else:
        test = 'mann-whitney'
        stat, p_value = stats.mannwhitneyu(global_ms, global_hv)
    
    if test == 'ttest':
        if p_value < 0.001:
            print(f"t-statistic: {stat}, p < 0.001")
        else:
            print(f"t-statistic: {stat}, p = {p_value:.3f}")

    elif test == 'mann-whitney':
        if p_value < 0.001:
            print(f"mann-whitney-statistic: {stat}, p < 0.001")
        else:
            print(f"mann-whitney-statistic: {stat}, p = {p_value:.3f}")


In [17]:
# Function to return global graph measures:

def return_global(data, measure='degree', weight='weight', centrality='mean', network='global', 
               layer='single'):
    data_list = calculate_graph_measures(data, measure=measure, weight=weight, layer=layer)
    
    if measure == 'degree' or network != 'global':
        if centrality == 'mean':
            global_list = [np.mean(list(s.values())) for s in data_list]
        elif centrality ==  'median':
            global_list = [np.median(list(s.values())) for s in data_list]
        else:
            raise ValueError(f"Unsupported centrality measure: {centrality}")
    elif (measure == 'node_degree' or
          measure == 'average_clustering' or
          measure == 'local_efficiency' or
          measure == 'global_efficiency' or
          measure == 'assortativity'or
          measure == 'clustering_coefficient'):
        global_list = [s for s in data_list]
    else:
        if centrality == 'mean':
            global_list = [mean(s) for s in data_list]
        elif centrality ==  'median':
            global_list = [median(s) for s in data_list]
        else:
            raise ValueError(f"Unsupported centrality measure: {centrality}")
            
    return global_list
    

In [18]:
# Independence test for nodal measures:

def stats_nodal_test(ms=ms_st, hv=hv_st, measure='degree', weight='weight', verbose=True, network='nodal', 
                     layer='single', sn=False):
    ms_list = calculate_graph_measures(ms, measure=measure, weight=weight, network=network)
    hv_list = calculate_graph_measures(hv, measure=measure, weight=weight, network=network)
            
    nodes_ms = [[d[key] for d in ms_list] for key in ms_list[0]]
    nodes_hv = [[d[key] for d in hv_list] for key in hv_list[0]]
    
    
    significant_nodes = []
    significant_nodes_measures_ms = []
    significant_nodes_measures_hv = []
    
    for i in range(len(nodes_ms)):
        _, p_value_shapiro_ms = stats.shapiro(nodes_ms)
        _, p_value_shapiro_hv = stats.shapiro(nodes_hv)
        
        if p_value_shapiro_ms > 0.05 and p_value_shapiro_hv > 0.05:
            test = 'ttest'
        else:
            test = 'mann-whitney'
        
        if test == 'ttest':
            stat, p_value = stats.ttest_ind(nodes_ms[i], nodes_hv[i])
        elif test == 'mann-whitney':
            stat, p_value = stats.mannwhitneyu(nodes_ms[i], nodes_hv[i])
        
        if verbose:
            if p_value < 0.001:
                print(f"statistic: {stat}, p < 0.001")
                print("  Significant")
            elif p_value < 0.05:
                print(f"statistic: {stat}, p = {p_value:.3f}")
                print("  Significant")
            else:
                print(f"statistic: {stat}, p = {p_value:.3f}")
                

        # Check if the p-value is less than 0.01
        if p_value < 0.001:
            significant_nodes.append(i+1)
            significant_nodes_measures_ms.append(nodes_ms[i])
            significant_nodes_measures_hv.append(nodes_hv[i])
            
    print(f"Amount of significant nodes: {len(significant_nodes)}/76")
    print(f"List of significant nodes: {significant_nodes}")
    print(f"Test used: {test}")
    
    significant_nodes_measures_ms = np.array(significant_nodes_measures_ms)
    significant_nodes_measures_hv = np.array(significant_nodes_measures_hv)
    
    summary_nodes = "{}/76".format(len(significant_nodes))
    
    if sn:
        return significant_nodes_measures_ms, significant_nodes_measures_hv, significant_nodes
    else:
        return significant_nodes_measures_ms, significant_nodes_measures_hv, summary_nodes

    

<a id='a4'></a>
<h1>Graph measures</h1>

Now we use the functions to calculate independence between groups using graph measures. Each measure affords a different kind of calculation, some are only global, some only nodal, some both. For nodal tests, we print out a list of the nodes where we find a significant difference between groups. For each section, we first check for differences between patients and controls, and under the subsection "Phenotype" we look for differences between rrms patients and ppms/spms patints.

<a id='a4.1'></a>
<h2>Strength</h2>

In [19]:
print("Statistical test for difference between global strengths (structural):")
stats_test()

Statistical test for difference between global strengths (structural):
mann-whitney-statistic: 393.0, p < 0.001


In [20]:
print("Statistical test for difference between global strengths (functional):")
stats_test(ms=ms_func, hv=hv_func)

Statistical test for difference between global strengths (functional):
mann-whitney-statistic: 1573.0, p = 0.192


In [21]:
print("Statistical test for difference between global strengths (combined):")
stats_test(ms=ms_combined, hv=hv_combined)

Statistical test for difference between global strengths (combined):
mann-whitney-statistic: 884.0, p = 0.022


In [22]:
print("Statistical test for nodal strength differences (structural):")
significant_nodes_ms_strength, significant_nodes_hv_strength, nodal_strength_st = stats_nodal_test(ms_st, hv_st, verbose=False)


Statistical test for nodal strength differences (structural):
Amount of significant nodes: 54/76
List of significant nodes: [2, 3, 4, 5, 6, 8, 9, 10, 11, 13, 14, 15, 17, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 32, 33, 36, 38, 39, 40, 43, 44, 45, 47, 48, 51, 53, 54, 55, 56, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74]
Test used: mann-whitney


/home/vant/anaconda3/envs/svm/lib/python3.11/site-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [23]:
nodal_strength_st

'54/76'

In [24]:
print("Statistical test for nodal strength differences (functional):")
stats_nodal_test(ms=ms_func, hv=hv_func, verbose=False)

Statistical test for nodal strength differences (functional):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

In [25]:
significant_nodes_ms_strength_combined, significant_nodes_hv_strength_combined, nodal_strength_comb = stats_nodal_test(ms=ms_combined, hv=hv_combined, verbose=False)

Amount of significant nodes: 5/76
List of significant nodes: [43, 45, 56, 59, 64]
Test used: mann-whitney


<h3>Phenotypes</h3>

In [26]:
print("Statistical test for difference between global strengths of phenotypes(structural):")
stats_test(ms=ms_rr, hv=ms_psp)

Statistical test for difference between global strengths of phenotypes(structural):
mann-whitney-statistic: 1573.0, p = 0.284


In [27]:
print("Statistical test for difference between global strengths of phenotypes(functional):")
stats_test(ms=ms_rr_func, hv=ms_psp_func)

Statistical test for difference between global strengths of phenotypes(functional):
mann-whitney-statistic: 1358.0, p = 0.929


In [28]:
print("Statistical test for nodal strength differences of phenotypes(structural):")
stats_nodal_test(ms=ms_rr, hv=ms_psp, verbose=False)

Statistical test for nodal strength differences of phenotypes(structural):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

In [29]:
print("Statistical test for nodal strength differences of phenotypes(functional):")
significant_strength_rr, significant_strength_psp, nodes_strength_phe = stats_nodal_test(ms=ms_rr_func, hv=ms_psp_func, verbose=False)

Statistical test for nodal strength differences of phenotypes(functional):
Amount of significant nodes: 1/76
List of significant nodes: [42]
Test used: mann-whitney


<a id='a4.2'></a>
<h2>Node Degree</h2>

In [30]:
print("Total Node Degrees for Each MS Graph:")
print(calculate_graph_measures(ms_st, measure='node_degree'))

Total Node Degrees for Each MS Graph:
[3266, 3488, 3412, 3450, 3460, 3474, 3396, 3440, 3446, 3478, 3486, 3478, 3594, 3434, 3472, 3396, 3594, 3482, 3438, 3450, 2922, 3392, 3410, 3486, 3506, 3490, 3452, 3390, 3356, 3460, 3482, 3478, 3478, 3468, 3524, 3412, 3512, 3466, 3482, 3390, 3394, 3386, 3400, 3462, 3468, 3454, 3594, 3468, 3430, 3500, 3466, 3402, 3442, 2756, 3380, 3480, 3348, 3222, 3494, 3596, 3210, 3332, 3430, 3376, 3470, 3594, 3444, 3480, 3594, 3182, 3396, 3452, 3594, 3466, 3430, 3594, 3488, 3594, 3594, 3442, 3458, 3504, 3404, 3372, 3202, 3248, 2870, 3060, 3164, 3240, 3254, 3012, 3272, 3158, 3274, 3196, 3222, 2860, 3178, 3050, 2462, 3268, 2920, 3296, 3160, 3054, 3228, 2448, 3346, 3050, 3420, 3328, 3186, 3082, 3146, 2998, 3260, 3120, 3000, 3342, 3262, 3158, 3294, 3242, 3250, 3176, 3056, 3110, 3390, 3194, 3382, 3036, 3254, 2552, 3326, 2994, 3170, 3330, 3268, 3306, 3184, 3158, 3008, 2870, 3066, 3066, 2878]


In [31]:
print("Total Node Degrees for Each HV Graph:")
print(calculate_graph_measures(hv_st, measure='node_degree'))

Total Node Degrees for Each HV Graph:
[3538, 3414, 3532, 3480, 3512, 3524, 3472, 3540, 3506, 3546, 3214, 3104, 3328, 3222, 3392, 3358, 3154, 3322]


In [32]:
print("Global node degree test (structural):")
stats_test(measure='node_degree')

Global node degree test (structural):
mann-whitney-statistic: 960.0, p = 0.058


In [33]:
print("Global node degree test (functional):")
stats_test(ms=ms_func, hv=hv_func, measure='node_degree')

Global node degree test (functional):
t-statistic: 2.9538359157980647, p = 0.004


In [34]:
print("Global node degree test (combined):")
stats_test(ms=ms_combined, hv=hv_combined, measure='node_degree')

Global node degree test (combined):
mann-whitney-statistic: 1295.5, p = 0.888


<h3>Phenotypes</h3>

In [35]:
print("Global node degree test for phenotypes (structural):")
stats_test(ms=ms_rr, hv=ms_psp, measure='node_degree')

Global node degree test for phenotypes (structural):
mann-whitney-statistic: 1619.0, p = 0.186


In [36]:
print("Global node degree test for phenotypes (functional):")
stats_test(ms=ms_rr_func, hv=ms_psp_func, measure='node_degree')

Global node degree test for phenotypes (functional):
t-statistic: 0.6178329223909763, p = 0.538


<a id='a4.3'></a>
<h2>Degree Centrality</h2>

In [37]:
print("Global degree centrality test (structural):")
stats_test(measure='degree_centrality')

Global degree centrality test (structural):
mann-whitney-statistic: 960.0, p = 0.058


In [38]:
print("Global degree centrality test (functional):")
stats_test(ms=ms_func, hv=hv_func, measure='degree_centrality')

Global degree centrality test (functional):
t-statistic: 2.9538359157980656, p = 0.004


In [39]:
print("Global degree centrality test (combined):")
stats_test(ms=ms_combined, hv=hv_combined, measure='degree_centrality')

Global degree centrality test (combined):
mann-whitney-statistic: 1297.0, p = 0.894


In [40]:
print("Statistical test for nodal degree centrality (structural):")
significant_nodes_ms_dc, significant_nodes_hv_dc, nodal_dc_st = stats_nodal_test(ms_st, hv_st, measure='degree_centrality', verbose=False)

Statistical test for nodal degree centrality (structural):
Amount of significant nodes: 3/76
List of significant nodes: [36, 40, 43]
Test used: mann-whitney


In [41]:
print("Statistical test for nodal degree centrality (functional):")
significant_nodes_ms_dc_func, significant_nodes_hv_dc_func, nodal_dc_func = stats_nodal_test(ms=ms_func, hv=hv_func, measure='degree_centrality', verbose=False)

Statistical test for nodal degree centrality (functional):
Amount of significant nodes: 1/76
List of significant nodes: [53]
Test used: mann-whitney


In [42]:
print("Statistical test for nodal degree centrality (combined):")
stats_nodal_test(ms=ms_combined, hv=hv_combined, measure='degree_centrality', verbose=False)

Statistical test for nodal degree centrality (combined):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

<h3>Phenotypes</h3>

In [43]:
print("Global degree centrality test for phenotypes(structural):")
stats_test(ms=ms_rr, hv=ms_psp, measure='degree_centrality')

Global degree centrality test for phenotypes(structural):
mann-whitney-statistic: 1619.0, p = 0.186


In [44]:
print("Global degree centrality test for phenotypes(functional):")
stats_test(ms=ms_rr_func, hv=ms_psp_func, measure='degree_centrality')

Global degree centrality test for phenotypes(functional):
t-statistic: 0.6178329223909796, p = 0.538


In [45]:
print("Statistical test for nodal degree centrality phenotypes (structural):")
stats_nodal_test(ms_rr, ms_psp, measure='degree_centrality', verbose=False)

Statistical test for nodal degree centrality phenotypes (structural):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

In [46]:
print("Statistical test for nodal degree centrality phenotypes (functional):")
stats_nodal_test(ms_rr_func, ms_psp_func, measure='degree_centrality', verbose=False)

Statistical test for nodal degree centrality phenotypes (functional):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

<a id='a4.4'></a>
<h2>Closeness centrality</h2>

In [47]:
print("Global closeness centrality test (structural):")
stats_test(measure='closeness_centrality')

Global closeness centrality test (structural):
mann-whitney-statistic: 975.0, p = 0.069


In [48]:
print("Global closeness centrality test (functional):")
stats_test(ms=ms_func, hv=hv_func, measure='closeness_centrality')

Global closeness centrality test (functional):
t-statistic: 3.0222420670603247, p = 0.003


In [49]:
stats_test(ms=ms_combined, hv=hv_combined, measure='closeness_centrality')

mann-whitney-statistic: 1289.0, p = 0.861


In [50]:
print("Statistical test for nodal closeness centrality (structural):")
significant_nodes_ms_cc, significant_nodes_hv_cc, nodal_cc_st = stats_nodal_test(measure='closeness_centrality', verbose=False)

Statistical test for nodal closeness centrality (structural):
Amount of significant nodes: 3/76
List of significant nodes: [36, 40, 43]
Test used: mann-whitney


In [51]:
print("Statistical test for nodal closeness centrality (functional):")
significant_nodes_ms_cc_func, significant_nodes_hv_cc_func, nodal_cc_func = stats_nodal_test(ms=ms_func, hv=hv_func, measure='closeness_centrality', verbose=False)

Statistical test for nodal closeness centrality (functional):
Amount of significant nodes: 1/76
List of significant nodes: [53]
Test used: mann-whitney


In [52]:
print("Statistical test for nodal closeness centrality (combined):")
stats_nodal_test(ms=ms_combined, hv=hv_combined, measure='closeness_centrality', verbose=False)

Statistical test for nodal closeness centrality (combined):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

<h3>Phenotypes</h3>

In [53]:
print("Global closeness centrality test (structural):")
stats_test(ms=ms_rr, hv=ms_psp, measure='closeness_centrality')

Global closeness centrality test (structural):
mann-whitney-statistic: 1621.0, p = 0.183


In [54]:
print("Global closeness centrality test (functional):")
stats_test(ms=ms_rr_func, hv=ms_psp_func, measure='closeness_centrality')

Global closeness centrality test (functional):
t-statistic: 0.5602840974310408, p = 0.576


In [55]:
print("Statistical test for nodal closeness centrality (structural):")
stats_nodal_test(ms=ms_rr, hv=ms_psp, measure='closeness_centrality', verbose=False)

Statistical test for nodal closeness centrality (structural):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

In [56]:
print("Statistical test for nodal closeness centrality (functional):")
stats_nodal_test(ms=ms_rr_func, hv=ms_psp_func, measure='closeness_centrality', verbose=False)

Statistical test for nodal closeness centrality (functional):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

<a id='a4.5'></a>
<h2>Clustering coefficient</h2>

In [57]:
print("Global clustering coefficient test (structural):")
stats_test(measure='average_clustering')

Global clustering coefficient test (structural):
mann-whitney-statistic: 1127.0, p = 0.307


In [58]:
print("Global clustering coefficient test (functional):")
stats_test(ms=ms_func, hv=hv_func, measure='average_clustering')

Global clustering coefficient test (functional):
t-statistic: 2.811489986718919, p = 0.006


In [59]:
stats_test(ms=ms_combined, hv=hv_combined, measure='average_clustering')

mann-whitney-statistic: 1310.0, p = 0.948


In [60]:
print("Statistical test for nodal clustering coefficient (structural):")
stats_nodal_test(measure='clustering', verbose=False)

Statistical test for nodal clustering coefficient (structural):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

In [61]:
print("Statistical test for nodal clustering coefficient (functional):")
significant_nodes_ms_cluster_func, significant_nodes_hv_cluster_func, nodal_clustering_func = stats_nodal_test(ms=ms_func, hv=hv_func, measure='clustering', verbose=False)

Statistical test for nodal clustering coefficient (functional):
Amount of significant nodes: 1/76
List of significant nodes: [30]
Test used: mann-whitney


In [62]:
print("Statistical test for nodal clustering coefficient (combined):")
stats_nodal_test(ms=ms_combined, hv=hv_combined, measure='clustering', verbose=False)

Statistical test for nodal clustering coefficient (combined):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

<h3>Phenotypes</h3>

In [63]:
print("Global clustering coefficient test (structural):")
stats_test(ms=ms_rr, hv=ms_psp, measure='average_clustering')

Global clustering coefficient test (structural):
mann-whitney-statistic: 1596.0, p = 0.231


In [64]:
print("Global clustering coefficient test (functional):")
stats_test(ms=ms_rr_func, hv=ms_psp_func, measure='average_clustering')

Global clustering coefficient test (functional):
t-statistic: 0.4862288390254382, p = 0.628


In [65]:
print("Statistical test for nodal clustering coefficient (structural):")
stats_nodal_test(ms=ms_rr, hv=ms_psp, measure='clustering', verbose=False)

Statistical test for nodal clustering coefficient (structural):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

In [66]:
print("Statistical test for nodal clustering coefficient (functional):")
stats_nodal_test(ms=ms_rr_func, hv=ms_psp_func, measure='clustering', verbose=False)

Statistical test for nodal clustering coefficient (functional):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

<a id='a4.6'></a>
<h2>Local Efficiency</h2>

In [67]:
print("Local efficiency test (structural):")
stats_test(measure='local_efficiency')

Local efficiency test (structural):
mann-whitney-statistic: 1127.0, p = 0.307


In [68]:
print("Local efficiency test (functional):")
stats_test(ms=ms_func, hv=hv_func, measure='local_efficiency')

Local efficiency test (functional):
t-statistic: 2.7751763076004203, p = 0.006


In [69]:
print("Local efficiency test (combined):")
stats_test(ms=ms_combined, hv=hv_combined, measure='local_efficiency')

Local efficiency test (combined):
mann-whitney-statistic: 1310.0, p = 0.948


<h3>Phenotypes</h3>

In [70]:
print("Local efficiency test (structural):")
stats_test(ms=ms_rr, hv=ms_psp, measure='local_efficiency')

Local efficiency test (structural):
mann-whitney-statistic: 1595.0, p = 0.233


In [71]:
print("Local efficiency test (functional):")
stats_test(ms=ms_rr_func, hv=ms_psp_func, measure='local_efficiency')

Local efficiency test (functional):
t-statistic: 0.5739715300555817, p = 0.567


<a id='a4.7'></a>
<h2>Global efficiency</h2>

In [72]:
print("Global efficiency test (structural):")
stats_test(measure='global_efficiency')

Global efficiency test (structural):
mann-whitney-statistic: 960.5, p = 0.058


In [73]:
print("Global efficiency test (functional):")
stats_test(ms=ms_func, hv=hv_func, measure='global_efficiency')

Global efficiency test (functional):
t-statistic: 3.0434820887042195, p = 0.003


In [74]:
print("Global efficiency test (combined):")
stats_test(ms=ms_combined, hv=hv_combined, measure='global_efficiency')

Global efficiency test (combined):
mann-whitney-statistic: 1295.5, p = 0.888


<h3>Phenotypes</h3>

In [75]:
print("Global efficiency test (structural):")
stats_test(ms=ms_rr, hv=ms_psp, measure='global_efficiency')

Global efficiency test (structural):
mann-whitney-statistic: 1620.0, p = 0.184


In [76]:
print("Global efficiency test (functional):")
stats_test(ms=ms_rr_func, hv=ms_psp_func, measure='global_efficiency')

Global efficiency test (functional):
t-statistic: 0.5972349699312437, p = 0.551


<a id='a4.8'></a>
<h2>Betweenness Centrality</h2>

In [77]:
print("Global betweenness centrality test (structural):")
stats_test(measure='betweenness_centrality')

Global betweenness centrality test (structural):
mann-whitney-statistic: 1686.5, p = 0.058


In [78]:
print("Global betweenness centrality test (functional):")
stats_test(ms=ms_func, hv=hv_func, measure='betweenness_centrality')

Global betweenness centrality test (functional):
t-statistic: -3.2044492356268512, p = 0.002


In [79]:
print("Global betweenness centrality test (combined):")
stats_test(ms=ms_combined, hv=hv_combined, measure='betweenness_centrality')

Global betweenness centrality test (combined):
mann-whitney-statistic: 1349.0, p = 0.894


In [80]:
print("Statistical nodal test for betweenness centrality (structural):")
significant_nodes_ms_bc, significant_nodes_hv_bc, nodal_bc_st = stats_nodal_test(measure='betweenness_centrality', verbose=False)

Statistical nodal test for betweenness centrality (structural):
Amount of significant nodes: 1/76
List of significant nodes: [45]
Test used: mann-whitney


In [81]:
print("Statistical nodal test for betweenness centrality (functional):")
significant_nodes_ms_bc_func, significant_nodes_hv_bc_func, nodal_bc_func = stats_nodal_test(ms=ms_func, hv=hv_func, measure='betweenness_centrality', verbose=False)

Statistical nodal test for betweenness centrality (functional):
Amount of significant nodes: 2/76
List of significant nodes: [59, 71]
Test used: mann-whitney


In [82]:
print("Statistical nodal test for betweenness centrality (combined):")
significant_nodes_ms_bc_combined, significant_nodes_hv_bc_combined, nodal_bc_comb = stats_nodal_test(ms=ms_combined, hv=hv_combined, measure='betweenness_centrality', verbose=False)

Statistical nodal test for betweenness centrality (combined):
Amount of significant nodes: 1/76
List of significant nodes: [6]
Test used: mann-whitney


<h3>Phenotypes</h3>

In [83]:
print("Global betweenness centrality test (structural):")
stats_test(ms=ms_rr, hv=ms_psp, measure='betweenness_centrality')

Global betweenness centrality test (structural):
mann-whitney-statistic: 1127.5, p = 0.180


In [84]:
print("Global betweenness centrality test (functional):")
stats_test(ms=ms_rr_func, hv=ms_psp_func, measure='betweenness_centrality')

Global betweenness centrality test (functional):
t-statistic: -0.5563181214287746, p = 0.579


In [85]:
print("Nodal test for betweenness centrality (structural):")
stats_nodal_test(ms=ms_rr, hv=ms_psp, measure='betweenness_centrality', verbose=False)

Nodal test for betweenness centrality (structural):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

In [86]:
print("Nodal test for betweenness centrality (functional):")
stats_nodal_test(ms=ms_rr_func, hv=ms_psp_func, measure='betweenness_centrality', verbose=False)

Nodal test for betweenness centrality (functional):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

<a id='a4.9'></a>
<h2>Eigenvector Centrality</h2>

In [87]:
print("Global eigenvector centrality test (structural):")
stats_test(measure='eigenvector_centrality')

Global eigenvector centrality test (structural):
mann-whitney-statistic: 807.0, p = 0.007


In [88]:
print("Global eigenvector centrality test (functional):")
stats_test(ms=ms_func, hv=hv_func, measure='eigenvector_centrality')

Global eigenvector centrality test (functional):
mann-whitney-statistic: 1615.0, p = 0.128


In [89]:
print("Global eigenvector centrality test (combined):")
stats_test(ms=ms_combined, hv=hv_combined, measure='eigenvector_centrality')

Global eigenvector centrality test (combined):
mann-whitney-statistic: 1039.0, p = 0.138


In [90]:
print("Statistical nodal test for eigenvector centrality (structural):")
significant_nodes_ms_ec, significant_nodes_hv_ec, nodal_ec_st = stats_nodal_test(measure='eigenvector_centrality', verbose=False)

Statistical nodal test for eigenvector centrality (structural):
Amount of significant nodes: 5/76
List of significant nodes: [36, 43, 46, 61, 64]
Test used: mann-whitney


In [91]:
print("Statistical nodal test for eigenvector centrality (functional):")
stats_nodal_test(ms=ms_func, hv=hv_func, measure='eigenvector_centrality', verbose=False)

Statistical nodal test for eigenvector centrality (functional):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

In [92]:
print("Statistical nodal test for eigenvector centrality (combined):")
stats_nodal_test(ms=ms_combined, hv=hv_combined, measure='eigenvector_centrality', verbose=False)

Statistical nodal test for eigenvector centrality (combined):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

<h3>Phenotypes</h3>

In [93]:
print("Global eigenvector centrality test (structural):")
stats_test(ms=ms_rr, hv=ms_psp, measure='eigenvector_centrality')

Global eigenvector centrality test (structural):
mann-whitney-statistic: 1642.0, p = 0.148


In [94]:
print("Global eigenvector centrality test (functional):")
stats_test(ms=ms_rr_func, hv=ms_psp_func, measure='eigenvector_centrality')

Global eigenvector centrality test (functional):
mann-whitney-statistic: 1026.0, p = 0.058


In [95]:
print("Statistical test for eigenvector centrality (structural):")
stats_nodal_test(ms=ms_rr, hv=ms_psp, measure='eigenvector_centrality', verbose=False)

Statistical test for eigenvector centrality (structural):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

In [96]:
print("Statistical test for eigenvector centrality (functional):")
stats_nodal_test(ms=ms_rr_func, hv=ms_psp_func, measure='eigenvector_centrality', verbose=False)

Statistical test for eigenvector centrality (functional):
Amount of significant nodes: 0/76
List of significant nodes: []
Test used: mann-whitney


(array([], dtype=float64), array([], dtype=float64), '0/76')

<a id='a4.10'></a>
<h2>Assortativity</h2>

In [97]:
print("Global assortativity test (structural):")
stats_test(measure='assortativity')

Global assortativity test (structural):
mann-whitney-statistic: 1628.0, p = 0.111


In [98]:
print("Global assortativity test (functional):")
stats_test(ms=ms_func, hv=hv_func, measure='assortativity')

Global assortativity test (functional):
mann-whitney-statistic: 896.0, p = 0.026


In [99]:
stats_test(ms=ms_combined, hv=hv_combined, measure='assortativity')

mann-whitney-statistic: 1576.0, p = 0.187


<h3>Phenotypes</h3>

In [100]:
print("Global assortativity test (structural):")
stats_test(ms=ms_rr, hv=ms_psp, measure='assortativity')

Global assortativity test (structural):
mann-whitney-statistic: 1225.0, p = 0.417


In [101]:
print("Global assortativity test (functional):")
stats_test(ms=ms_rr_func, hv=ms_psp_func, measure='assortativity')

Global assortativity test (functional):
mann-whitney-statistic: 1559.0, p = 0.319


<a id='a4.11'></a>
<h2>Rich Club</h2>

In [102]:
print("Global rich club test (structural):")
stats_test(measure='rich_club')

Global rich club test (structural):
mann-whitney-statistic: 870.5, p = 0.017


In [103]:
print("Global rich club test (functional):")
stats_test(ms=ms_func, hv=hv_func, measure='rich_club')

Global rich club test (functional):
t-statistic: 3.444584978966663, p < 0.001


In [104]:
print("Global rich club test (combined):")
stats_test(ms=ms_combined, hv=hv_combined, measure='rich_club')

Global rich club test (combined):
mann-whitney-statistic: 1205.5, p = 0.527


<h3>Phenotypes</h3>

In [105]:
print("Global rich club test (structural):")
stats_test(ms=ms_rr, hv=ms_psp, measure='rich_club')

Global rich club test (structural):
mann-whitney-statistic: 1651.0, p = 0.132


In [106]:
print("Global rich club test (functional):")
stats_test(ms=ms_rr_func, hv=ms_psp_func, measure='rich_club')

Global rich club test (functional):
t-statistic: 1.3337455970759524, p = 0.184


<a id='a4.11.1'></a>
<h3>Results</h3>

From the global graph measures, only strength obtains a significance p<0.001 when comparing the structural networks of patients and controls. Rich Club also is p<0.001, but when using the functional networks. There is no global measure where we obtain a significant difference between phenotype groups.

As to the nodal measures, we have been obtaining lists of nodes with significant differences between patients and controls. Here, we summarize the amount of nodes with significant differences for each of the nodal graph measures.

In [107]:
table_nodes = pd.DataFrame()

table_nodes['strength'] = [nodal_strength_st, 'n.a', nodal_strength_comb]
table_nodes['degree centrality'] = [nodal_dc_st, nodal_dc_func, 'n.a']
table_nodes['closeness  centrality'] = [nodal_cc_st, nodal_cc_func, 'n.a.']
table_nodes['clustering coefficient'] = ['n.a.', nodal_clustering_func, 'n.a.']
table_nodes['betweenness centrality'] = [nodal_bc_st, nodal_bc_func, nodal_bc_comb]
table_nodes['eigenvector centrality'] = [nodal_ec_st, 'n.a.', 'n.a.']

table_nodes['type'] = ['structural', 'functional', 'combined']
table_nodes = table_nodes.set_index('type')
table_nodes.index.name=None

In [108]:
print("Amount of nodes with significant differences between patients and hv controls:")
table_nodes

Amount of nodes with significant differences between patients and hv controls:


,strength,degree centrality,closeness centrality,clustering coefficient,betweenness centrality,eigenvector centrality
structural,54/76,3/76,3/76,n.a.,1/76,5/76
functional,n.a,1/76,1/76,1/76,2/76,n.a.
combined,5/76,n.a,n.a.,n.a.,1/76,n.a.


<a id='a5'></a>
<h1>Classification</h1>

In [109]:
from sklearn.model_selection import train_test_split, GridSearchCV
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, precision_score, recall_score, classification_report, f1_score
from sklearn.feature_selection import RFE
from imblearn.over_sampling import SMOTE
from sklearn.utils import shuffle
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold
from scipy.stats import ttest_ind

<a id='a5.1'></a>
<h2>Functions for FA classification</h2>

In [110]:
# Function for classification, including grid search for best parameters and SMOTE for data augmentation 

def classify_oversampled(data, connections=False):
    
    triuim1 = np.triu_indices_from(st_matrices[0], k=1)
    X = [np.array(matrix)[triuim1] for matrix in data]

    # we convert the list of upper triangles to a 2D array
    X = np.array(X)
    y = np.array(labels)
    
    # we use SMOTE for oversampling
    smote = SMOTE(sampling_strategy='auto', random_state=42)

    # variables to store the best results
    best_accuracy = 0.0
    best_precision = 0.0
    best_recall = 0.0
    best_f1 = 0.0
    best_confusion_matrix = None
    best_params = None

    num_instances = 50

    progress_bar = tqdm(total=num_instances, desc="Instances", position=0, leave=True)


    for instance in range(num_instances):
        
        X_resampled, y_resampled = smote.fit_resample(X, y)

        X_resampled, y_resampled = shuffle(X_resampled, y_resampled, random_state=42)

        X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=24)

        clf = SVC()

        # Grid search 
        param_grid = {"C": [0.01, 0.1, 1, 10, 100, 200], "gamma": [0.001, 0.01, 0.1, 1, 10]}
        grid_search = GridSearchCV(clf, param_grid=param_grid, cv=11)
        grid_search.fit(X_train, y_train)

        # Train 
        best_clf = SVC(C=grid_search.best_params_['C'], gamma=grid_search.best_params_['gamma'])
        best_clf.fit(X_train, y_train)

        # Predictions 
        preds = best_clf.predict(X_test)

        # Evaluation
        accuracy = accuracy_score(y_test, preds)
        precision = precision_score(y_test, preds)
        recall = recall_score(y_test, preds)
        f1 = f1_score(y_test, preds)
        cnf_matrix = confusion_matrix(y_test, preds)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_precision = precision
            best_recall = recall
            best_f1 = f1
            best_confusion_matrix = cnf_matrix
            best_params = grid_search.best_params_

            
        progress_bar.update(1)

    
    progress_bar.close()

    # Printing the best results
    print("Best Results:")
    print("Best Accuracy: {:.4f}".format(best_accuracy))
    print("Best Precision: {:.4f}".format(best_precision))
    print("Best Recall: {:.4f}".format(best_recall))
    print("Best F1 Score: {:.4f}".format(best_f1))
    print("Best Confusion Matrix:")
    print(best_confusion_matrix)
    print("Best Parameters (C and Gamma):", best_params)
    
    if connections: 
        
        X_resampled, y_resampled = smote.fit_resample(X, y)

        X_resampled, y_resampled = shuffle(X_resampled, y_resampled, random_state=42)

        X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=24)

        # Train the final model with the best hyperparameters
        final_clf = SVC(C=best_params['C'], gamma=best_params['gamma'], kernel='rbf')
        final_clf.fit(X_train, y_train)

        # Predictions
        preds = final_clf.predict(X_test)

        # Evaluation
        accuracy = accuracy_score(y_test, preds)
        precision = precision_score(y_test, preds)
        recall = recall_score(y_test, preds)
        cnf_matrix = confusion_matrix(y_test, preds)
        
        n_sv = final_clf.support_vectors_.shape[0]
        feature_importances = np.abs(np.dot(final_clf.dual_coef_, final_clf.support_vectors_)).flatten() / n_sv

        top_indices = np.argsort(feature_importances)[-7:][::-1]

        # Get the row and column indices for the flattened indices
        row_indices, col_indices = np.triu_indices(76, k=1)

        # Map the flattened indices to the original matrix indices
        top_original_indices = list(zip(row_indices[top_indices], col_indices[top_indices]))

        # Printing the result
        for idx, original_idx in enumerate(top_original_indices):
            print(f"Node connection: {original_idx[0] + 1, original_idx[1] + 1}, Importance: {feature_importances[top_indices[idx]]:.4f}")


In [111]:
# Function for classification, as above, and including stratified k fold validation 

def classify_oversampled_skf2(data, connections=False, title=None):
    triuim1 = np.triu_indices_from(st_matrices[0], k=1)
    X = [np.array(matrix)[triuim1] for matrix in data]

    # we convert the list of upper triangles to a 2D array
    X = np.array(X)
    y = np.array(labels)
    
    # we use SMOTE for oversampling
    smote = SMOTE(sampling_strategy='auto', random_state=42)

    # StratifiedKFold with n_splits=5
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    best_params = None

    # SVM model
    clf = SVC()

    # Grid search 
    param_grid = {"C": [0.01, 0.1, 1, 10, 100, 200], "gamma": [0.001, 0.01, 0.1, 1, 10]}
    grid_search = GridSearchCV(clf, param_grid=param_grid, cv=11)
    grid_search.fit(X, y)

    # we use the best hyperparameters for k-fold cross-validation
    best_params = grid_search.best_params_

    # variables to store the best results
    best_accuracy = 0.0
    best_precision = 0.0
    best_recall = 0.0
    best_f1 = 0.0
    best_confusion_matrix = None

    progress_bar = tqdm(total=skf.get_n_splits(X, y), desc="Folds", position=0, leave=True)

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # we apply SMOTE to the training data
        X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

        X_resampled, y_resampled = shuffle(X_resampled, y_resampled, random_state=42)

        # we train the model with the best parameters
        best_clf = SVC(C=best_params['C'], gamma=best_params['gamma'])
        best_clf.fit(X_resampled, y_resampled)

        # Predictions 
        preds = best_clf.predict(X_test)

        # Evaluation
        accuracy = accuracy_score(y_test, preds)
        precision = precision_score(y_test, preds)
        recall = recall_score(y_test, preds)
        f1 = f1_score(y_test, preds)
        cnf_matrix = confusion_matrix(y_test, preds)

        if accuracy > best_accuracy:
            best_accuracy = round(accuracy, 2)
            best_precision = round(precision, 2)
            best_recall = round(recall, 2)
            best_f1 = round(f1, 2)
            best_confusion_matrix = cnf_matrix

        progress_bar.update(1)

    progress_bar.close()

    # Printing the best results
    print("Best results {}:".format(title))
    print("Best Accuracy: {:.4f}".format(best_accuracy))
    print("Best Precision: {:.4f}".format(best_precision))
    print("Best Recall: {:.4f}".format(best_recall))
    print("Best F1 Score: {:.4f}".format(best_f1))
    print("Best Confusion Matrix:")
    print(best_confusion_matrix)
    print("Best Parameters (C and Gamma):", best_params)
    
    if connections: 
        # we apply SMOTE to the data
        X_resampled, y_resampled = smote.fit_resample(X, y)

        X_resampled, y_resampled = shuffle(X_resampled, y_resampled, random_state=42)

        X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=24)

        # we train the final model with the best hyperparameters
        final_clf = SVC(C=best_params['C'], gamma=best_params['gamma'], kernel='rbf')
        final_clf.fit(X_train, y_train)

        # Predictions 
        preds = final_clf.predict(X_test)

        # Evaluation
        accuracy = accuracy_score(y_test, preds)
        precision = precision_score(y_test, preds)
        recall = recall_score(y_test, preds)
        cnf_matrix = confusion_matrix(y_test, preds)
        
        n_sv = final_clf.support_vectors_.shape[0]
        feature_importances = np.abs(np.dot(final_clf.dual_coef_, final_clf.support_vectors_)).flatten() / n_sv

        top_indices = np.argsort(feature_importances)[-7:][::-1]

        # row and column indices for the flattened indices
        row_indices, col_indices = np.triu_indices(76, k=1)

        # we map the flattened indices to the original matrix indices
        top_original_indices = list(zip(row_indices[top_indices], col_indices[top_indices]))

        # Printing the result
        for idx, original_idx in enumerate(top_original_indices):
            print(f"Node connection: {original_idx[0] + 1, original_idx[1] + 1}, Importance: {feature_importances[top_indices[idx]]:.4f}")


    return best_accuracy, best_precision, best_recall, best_f1


<a id='a5.2'></a>
<h2>Connectivity based classification</h2>

Stratified k-fold validation was introduced late in the analysis process. For archiving purposes, we are including here the results we obtained previous to the use of k-fold validation. Results (only the ones validated with k-fold) will be summarized again on the results section.

In [112]:
# Classification using structural matrices FA connectivity
classify_oversampled(st_matrices, connections=True)

Instances: 100%|████████████████████████████████| 50/50 [08:08<00:00,  9.76s/it]

Best Results:
Best Accuracy: 0.9730
Best Precision: 0.9487
Best Recall: 1.0000
Best F1 Score: 0.9737
Best Confusion Matrix:
[[35  2]
 [ 0 37]]
Best Parameters (C and Gamma): {'C': 1, 'gamma': 0.1}
Node connection: (1, 55), Importance: 0.0312
Node connection: (12, 43), Importance: 0.0293
Node connection: (44, 74), Importance: 0.0274
Node connection: (36, 45), Importance: 0.0273
Node connection: (31, 43), Importance: 0.0269
Node connection: (11, 73), Importance: 0.0245
Node connection: (14, 39), Importance: 0.0243


In [113]:
# Classification using structural matrices FA connectivity and including stratified k-fold validation
a_st, p_st, r_st, f_st = classify_oversampled_skf2(data=st_matrices, connections=True, title="for structural FA connectivity")

Folds: 100%|██████████████████████████████████████| 5/5 [00:00<00:00, 28.89it/s]


Best results for structural FA connectivity:
Best Accuracy: 1.0000
Best Precision: 1.0000
Best Recall: 1.0000
Best F1 Score: 1.0000
Best Confusion Matrix:
[[ 3  0]
 [ 0 30]]
Best Parameters (C and Gamma): {'C': 200, 'gamma': 0.001}
Node connection: (12, 43), Importance: 3.2681
Node connection: (1, 55), Importance: 2.8923
Node connection: (44, 74), Importance: 2.5050
Node connection: (5, 20), Importance: 2.4635
Node connection: (38, 70), Importance: 2.3099
Node connection: (28, 45), Importance: 2.0751
Node connection: (3, 41), Importance: 1.9891


In [114]:
# Classification using functional connectivity matrices
classify_oversampled(func_matrices, connections=True)

Instances: 100%|████████████████████████████████| 50/50 [09:34<00:00, 11.49s/it]

Best Results:
Best Accuracy: 1.0000
Best Precision: 1.0000
Best Recall: 1.0000
Best F1 Score: 1.0000
Best Confusion Matrix:
[[37  0]
 [ 0 37]]
Best Parameters (C and Gamma): {'C': 1, 'gamma': 0.01}
Node connection: (7, 43), Importance: 0.0456
Node connection: (62, 69), Importance: 0.0426
Node connection: (10, 69), Importance: 0.0410
Node connection: (20, 61), Importance: 0.0406
Node connection: (17, 57), Importance: 0.0401
Node connection: (22, 61), Importance: 0.0391
Node connection: (17, 69), Importance: 0.0390


In [115]:
# Classification using functional connectivity matrices and including k-fold validation
a_func, p_func, r_func, f_func = classify_oversampled_skf2(data = func_matrices, connections=True,
                                    title="for functional connectivity")

Folds: 100%|██████████████████████████████████████| 5/5 [00:00<00:00, 24.83it/s]

Best results for functional connectivity:
Best Accuracy: 0.9400
Best Precision: 0.9400
Best Recall: 1.0000
Best F1 Score: 0.9700
Best Confusion Matrix:
[[ 1  2]
 [ 0 30]]
Best Parameters (C and Gamma): {'C': 100, 'gamma': 0.001}
Node connection: (7, 43), Importance: 0.5784
Node connection: (5, 38), Importance: 0.4736
Node connection: (21, 23), Importance: 0.4631
Node connection: (33, 69), Importance: 0.4550
Node connection: (20, 61), Importance: 0.4488
Node connection: (7, 38), Importance: 0.4390
Node connection: (17, 31), Importance: 0.4380


In [116]:
# Classification using functional and structural matrices combined
classify_oversampled(all_combined, connections=True)

Instances: 100%|████████████████████████████████| 50/50 [08:54<00:00, 10.69s/it]

Best Results:
Best Accuracy: 0.9865
Best Precision: 0.9737
Best Recall: 1.0000
Best F1 Score: 0.9867
Best Confusion Matrix:
[[36  1]
 [ 0 37]]
Best Parameters (C and Gamma): {'C': 1, 'gamma': 0.1}
Node connection: (31, 43), Importance: 0.0242
Node connection: (7, 43), Importance: 0.0222
Node connection: (14, 39), Importance: 0.0217
Node connection: (44, 74), Importance: 0.0196
Node connection: (45, 64), Importance: 0.0188
Node connection: (12, 43), Importance: 0.0172
Node connection: (45, 56), Importance: 0.0168


In [117]:
# Classification using functional and structural matrices combined and including k-fold validation
a_comb, p_comb, r_comb, f_comb = classify_oversampled_skf2(all_combined, connections=True, 
                                                           title='for weighted combination of matrices')

Folds: 100%|██████████████████████████████████████| 5/5 [00:00<00:00, 22.80it/s]

Best results for weighted combination of matrices:
Best Accuracy: 0.9700
Best Precision: 0.9700
Best Recall: 1.0000
Best F1 Score: 0.9800
Best Confusion Matrix:
[[ 2  1]
 [ 0 30]]
Best Parameters (C and Gamma): {'C': 100, 'gamma': 0.01}
Node connection: (31, 43), Importance: 0.2033
Node connection: (44, 74), Importance: 0.1716
Node connection: (7, 43), Importance: 0.1585
Node connection: (1, 69), Importance: 0.1514
Node connection: (15, 43), Importance: 0.1445
Node connection: (14, 39), Importance: 0.1437
Node connection: (12, 43), Importance: 0.1385


In [118]:
mix = pd.MultiIndex.from_product([['Connectivity'], ['structural', 'functional', 'combined']])
table_connectivity = pd.DataFrame(columns=mix) 

table_connectivity['statistic'] = ['accuracy', 'precision', 'recall', 'F1']

table_connectivity[('Connectivity', 'structural')] = [a_st, p_st, r_st, f_st]
table_connectivity[('Connectivity', 'functional')] = [a_func, p_func, r_func, f_func]
table_connectivity[('Connectivity', 'combined')] = [a_comb, p_comb, r_comb, f_comb]
table_connectivity = table_connectivity.set_index('statistic')
table_connectivity.index.name=None

<a id='a5.2.1'></a>
<h3>Results</h3>

In [119]:
print("Results of connectivity based classification:")
table_connectivity

Results of connectivity based classification:


Connectivity                    
            structural functional combined
accuracy           1.0       0.94     0.97
precision          1.0       0.94     0.97
recall             1.0       1.00     1.00
F1                 1.0       0.97     0.98

<a id='a5.3'></a>
<h2>Functions for graph based classification</h2>

In [120]:
# Function to classify using graph measures

def classify_graph(data, labels):

    X = np.array(data)
    y = np.array(labels)
    
    # we use SMOTE for oversampling
    smote = SMOTE(sampling_strategy='auto', random_state=42)

    # variables for results
    best_accuracy = 0.0
    best_precision = 0.0
    best_recall = 0.0
    best_f1 = 0.0
    best_confusion_matrix = None
    best_params = None

    num_instances = 50

    progress_bar = tqdm(total=num_instances, desc="Instances", position=0, leave=True)


    for instance in range(num_instances):
        
        X_resampled, y_resampled = smote.fit_resample(X, y)
        
        X_resampled, y_resampled = shuffle(X_resampled, y_resampled, random_state=42)

        X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=24)

        clf = SVC()

        # Grid search 
        param_grid = {"C": [0.01, 0.1, 1, 10, 100, 200], "gamma": [0.001, 0.01, 0.1, 1, 10]}
        grid_search = GridSearchCV(clf, param_grid=param_grid, cv=11)
        grid_search.fit(X_train, y_train)

        # Train 
        best_clf = SVC(C=grid_search.best_params_['C'], gamma=grid_search.best_params_['gamma'])
        best_clf.fit(X_train, y_train)

        # Predictions 
        preds = best_clf.predict(X_test)

        # Evaluation
        accuracy = accuracy_score(y_test, preds)
        precision = precision_score(y_test, preds)
        recall = recall_score(y_test, preds)
        f1 = f1_score(y_test, preds)
        cnf_matrix = confusion_matrix(y_test, preds)

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_precision = precision
            best_recall = recall
            best_f1 = f1
            best_confusion_matrix = cnf_matrix
            best_params = grid_search.best_params_

        progress_bar.update(1)

    progress_bar.close()

    # Printing the best results
    print("Best Results:")
    print("Best Accuracy: {:.4f}".format(best_accuracy))
    print("Best Precision: {:.4f}".format(best_precision))
    print("Best Recall: {:.4f}".format(best_recall))
    print("Best F1 Score: {:.4f}".format(best_f1))
    print("Best Confusion Matrix:")
    print(best_confusion_matrix)
    print("Best Parameters (C and Gamma):", best_params)

In [121]:
# Function to classify using graph measures including stratified k-fold validation

def classify_graph_skf(data, labels, title=None):
    X = np.array(data)
    y = np.array(labels)
    
    # we use SMOTE for oversampling
    smote = SMOTE(sampling_strategy='auto', random_state=42)

    # variables to store the best results
    best_accuracy = 0.0
    best_precision = 0.0
    best_recall = 0.0
    best_f1 = 0.0
    best_confusion_matrix = None
    best_params = None

    # stratifiedKFold with n_splits=5
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    progress_bar = tqdm(total=skf.get_n_splits(X, y), desc="Folds", position=0, leave=True)

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

        X_resampled, y_resampled = shuffle(X_resampled, y_resampled, random_state=42)

        clf = SVC()

        # Grid search 
        param_grid = {"C": [0.01, 0.1, 1, 10, 100, 200], "gamma": [0.001, 0.01, 0.1, 1, 10]}
        grid_search = GridSearchCV(clf, param_grid=param_grid, cv=11)
        grid_search.fit(X_resampled, y_resampled)

        # Train 
        best_clf = SVC(C=grid_search.best_params_['C'], gamma=grid_search.best_params_['gamma'])
        best_clf.fit(X_resampled, y_resampled)

        # Predictions
        preds = best_clf.predict(X_test)

        # Evaluation
        accuracy = accuracy_score(y_test, preds)
        precision = precision_score(y_test, preds, zero_division=1)
        recall = recall_score(y_test, preds)
        f1 = f1_score(y_test, preds)
        cnf_matrix = confusion_matrix(y_test, preds)

        if accuracy > best_accuracy:
            best_accuracy = round(accuracy, 2)
            best_precision = round(precision, 2)
            best_recall = round(recall, 2)
            best_f1 = round(f1, 2)
            best_confusion_matrix = cnf_matrix
            best_params = grid_search.best_params_

        
        progress_bar.update(1)

    progress_bar.close()

    # Printing the best results
    print("Best Results {}:".format(title))
    print("Best Accuracy: {:.4f}".format(best_accuracy))
    print("Best Precision: {:.4f}".format(best_precision))
    print("Best Recall: {:.4f}".format(best_recall))
    print("Best F1 Score: {:.4f}".format(best_f1))
    print("Best Confusion Matrix:")
    print(best_confusion_matrix)
    print("Best Parameters (C and Gamma):", best_params)
    
    return best_accuracy, best_precision, best_recall, best_f1


<a id='a5.4'></a>
<h2>Classification using global graph measures</h2>

In [122]:
# We generate labels 

zeros = [0] * 147
ones = [1] * 18

concat_labels = zeros + ones

In [123]:
global_strength_ms_nodes = calculate_graph_measures(ms_st)
global_strength_hv_nodes = calculate_graph_measures(hv_st)
global_strength_list = [list(i.values()) for i in global_strength_ms_nodes] + [list(i.values()) for i in global_strength_hv_nodes]

In [124]:
len(global_strength_hv_nodes)

18

In [125]:
classify_graph(data=global_strength_list, labels=concat_labels)

Instances: 100%|████████████████████████████████| 50/50 [00:51<00:00,  1.04s/it]

Best Results:
Best Accuracy: 0.9865
Best Precision: 0.9737
Best Recall: 1.0000
Best F1 Score: 0.9867
Best Confusion Matrix:
[[36  1]
 [ 0 37]]
Best Parameters (C and Gamma): {'C': 10, 'gamma': 0.01}


In [126]:
a_str, p_str, r_str, f1_str = classify_graph_skf(data=global_strength_list, 
                               labels=concat_labels, title='using global strength')

Folds: 100%|██████████████████████████████████████| 5/5 [00:05<00:00,  1.17s/it]

Best Results using global strength:
Best Accuracy: 0.9700
Best Precision: 0.7500
Best Recall: 1.0000
Best F1 Score: 0.8600
Best Confusion Matrix:
[[29  1]
 [ 0  3]]
Best Parameters (C and Gamma): {'C': 10, 'gamma': 0.01}


In [127]:
# We use the rich club measures obtained from functional matrices for another classification task

global_rich_ms_nodes = calculate_graph_measures(ms_func, measure='rich_club')
global_rich_hv_nodes = calculate_graph_measures(hv_func, measure='rich_club')
global_rich_data = [mean(i) for i in global_rich_ms_nodes] + [mean(i) for i in global_rich_hv_nodes]  

In [128]:
global_rich_data_l = [[i] for i in global_rich_data]

In [129]:
classify_graph(data=global_rich_data_l, labels=concat_labels)

Instances: 100%|████████████████████████████████| 50/50 [00:44<00:00,  1.11it/s]

Best Results:
Best Accuracy: 0.7703
Best Precision: 0.8333
Best Recall: 0.6757
Best F1 Score: 0.7463
Best Confusion Matrix:
[[32  5]
 [12 25]]
Best Parameters (C and Gamma): {'C': 100, 'gamma': 10}


In [130]:
a_ri, p_ri, r_ri, f1_ri = classify_graph_skf(data=global_rich_data_l, 
                                             labels=concat_labels, title='using rich club measure')

Folds: 100%|██████████████████████████████████████| 5/5 [00:04<00:00,  1.04it/s]

Best Results using rich club measure:
Best Accuracy: 0.9400
Best Precision: 0.6700
Best Recall: 0.6700
Best F1 Score: 0.6700
Best Confusion Matrix:
[[29  1]
 [ 1  2]]
Best Parameters (C and Gamma): {'C': 200, 'gamma': 10}


<a id='a5.5'></a>
<h2>Classification using vectors of diverse nodal graph measures</h2>

In the following cells, we take all the nodal graph measures that have obtained significant differences between patients and controls, and for each patient and control we concatenate the values of only the significant nodes to create a vector. So, each patient is characterized by a vector of values representing different nodal graph measures. We use these vectors to classify patients and controls, obtaining a high accuracy.

In [131]:
# We create the vector that gathers the different graph nodal measures

ms_strength_nodes = np.transpose(significant_nodes_ms_strength).tolist()
hv_strength_nodes = np.transpose(significant_nodes_hv_strength).tolist()

ms_degree_centrality_nodes = np.transpose(significant_nodes_ms_dc).tolist()
hv_degree_centrality_nodes = np.transpose(significant_nodes_hv_dc).tolist()

ms_degree_centrality_nodes_func = np.transpose(significant_nodes_ms_dc_func).tolist()
hv_degree_centrality_nodes_func = np.transpose(significant_nodes_hv_dc_func).tolist()

ms_closeness_centrality_nodes = np.transpose(significant_nodes_ms_cc).tolist()
hv_closeness_centrality_nodes = np.transpose(significant_nodes_hv_cc).tolist()

ms_closeness_centrality_nodes_func = np.transpose(significant_nodes_ms_cc_func).tolist()
hv_closeness_centrality_nodes_func = np.transpose(significant_nodes_hv_cc_func).tolist()

ms_clustering_coefficient_nodes_func = np.transpose(significant_nodes_ms_cluster_func).tolist()
hv_clustering_coefficient_nodes_func = np.transpose(significant_nodes_hv_cluster_func).tolist()

ms_betweenness_centrality_nodes = np.transpose(significant_nodes_ms_bc).tolist()
hv_betweenness_centrality_nodes = np.transpose(significant_nodes_hv_bc).tolist()

ms_betweenness_centrality_nodes_func = np.transpose(significant_nodes_ms_bc_func).tolist()
hv_betweenness_centrality_nodes_func = np.transpose(significant_nodes_hv_bc_func).tolist()

ms_eigenvector_centrality_nodes = np.transpose(significant_nodes_ms_ec).tolist()
hv_eigenvector_centrality_nodes = np.transpose(significant_nodes_hv_ec).tolist()

In [132]:
ms_nodes = [a + b + c + d + e + f + g + h + i  for a, b, c, d, e, f, g, h, i in zip(ms_strength_nodes, 
ms_degree_centrality_nodes, ms_degree_centrality_nodes_func, ms_closeness_centrality_nodes, 
ms_closeness_centrality_nodes_func, ms_clustering_coefficient_nodes_func, ms_betweenness_centrality_nodes, 
ms_betweenness_centrality_nodes_func, ms_eigenvector_centrality_nodes)]
print(len(ms_nodes))

hv_nodes = [a + b + c + d + e + f + g + h + i  for a, b, c, d, e, f, g, h, i in zip(hv_strength_nodes, 
hv_degree_centrality_nodes, hv_degree_centrality_nodes_func, hv_closeness_centrality_nodes, 
hv_closeness_centrality_nodes_func, hv_clustering_coefficient_nodes_func, hv_betweenness_centrality_nodes, 
hv_betweenness_centrality_nodes_func, hv_eigenvector_centrality_nodes)]
print(len(hv_nodes))

147
18


In [133]:
nodal_graph_data = ms_nodes + hv_nodes

In [134]:
classify_graph(data=nodal_graph_data, labels=concat_labels)

Instances: 100%|████████████████████████████████| 50/50 [00:52<00:00,  1.05s/it]

Best Results:
Best Accuracy: 0.9730
Best Precision: 0.9487
Best Recall: 1.0000
Best F1 Score: 0.9737
Best Confusion Matrix:
[[35  2]
 [ 0 37]]
Best Parameters (C and Gamma): {'C': 10, 'gamma': 0.01}


In [135]:
a_n, p_n, r_n, f1_n = classify_graph_skf(data=nodal_graph_data, 
                   labels=concat_labels, title='using vector of mixed nodal measures')

Folds: 100%|██████████████████████████████████████| 5/5 [00:05<00:00,  1.11s/it]

Best Results using vector of mixed nodal measures:
Best Accuracy: 0.9700
Best Precision: 0.7500
Best Recall: 1.0000
Best F1 Score: 0.8600
Best Confusion Matrix:
[[29  1]
 [ 0  3]]
Best Parameters (C and Gamma): {'C': 10, 'gamma': 0.01}


In [136]:
mix_graph = pd.MultiIndex.from_product([['Graph'], ["structural strength", 'functional rich club', 
                                                    'vector of node measures']])
table_graph = pd.DataFrame(columns=mix_graph) 

table_graph['statistic'] = ['accuracy', 'precision', 'recall', 'F1']
table_graph[('Graph', "structural strength")] = [a_str, p_str, r_str, f1_str]
table_graph[('Graph', 'functional rich club')] = [a_ri, p_ri, r_ri, f1_ri]
table_graph[('Graph', 'vector of node measures')] = [a_n, p_n, r_n, f1_n]
table_graph = table_graph.set_index('statistic')
table_graph.index.name=None

<a id='a5.5.1'></a>
<h3>Results</h3>

In [137]:
table_graph

Graph                                             
          structural strength functional rich club vector of node measures
accuracy                 0.97                 0.94                    0.97
precision                0.75                 0.67                    0.75
recall                   1.00                 0.67                    1.00
F1                       0.86                 0.67                    0.86

<a id='a5.6'></a>
<h2>Classification using brain volumes</h2>

<a id='a5.6.1'></a>
<h3>All brain volumes</h3>

In [138]:
# We normalize volumes from 0 to 1

flattened_list = [item for sublist in volumes for item in sublist]

global_min = min(flattened_list)
global_max = max(flattened_list)

print(global_min)
print(global_max)

normalized_list = [(x - global_min) / (global_max - global_min) for x in flattened_list]

num_elements = len(volumes[0])
normalized_volumes = [normalized_list[i:i + num_elements] for i in range(0, len(normalized_list), num_elements)]

print(min([min(i) for i in normalized_volumes]))
print(max([max(i) for i in normalized_volumes]))

40
40960
0.0
1.0


In [139]:
normalized_v_ms = normalized_volumes[:147]
normalized_v_hv = normalized_volumes[147:]
print(len(normalized_v_ms))
print(len(normalized_v_hv))

147
18


In [140]:
# we perform a t-test for independence between patients and controls

flat_list1 = [item for sublist in normalized_v_ms for item in sublist]
flat_list2 = [item for sublist in normalized_v_hv for item in sublist]

# Perform t-test
t_statistic, p_value = ttest_ind(np.array(flat_list1), np.array(flat_list2))

# Print the results
print(f'T-statistic: {t_statistic}')
print(f'P-value: {p_value}')

T-statistic: -3.678241051937038
P-value: 0.00023583029405776083


In [141]:
classify_graph(data=normalized_volumes, labels=concat_labels)

Instances: 100%|████████████████████████████████| 50/50 [00:48<00:00,  1.02it/s]

Best Results:
Best Accuracy: 0.9459
Best Precision: 0.9024
Best Recall: 1.0000
Best F1 Score: 0.9487
Best Confusion Matrix:
[[33  4]
 [ 0 37]]
Best Parameters (C and Gamma): {'C': 100, 'gamma': 10}


In [142]:
a_vol, p_vol, r_vol, f1_vol = classify_graph_skf(data=normalized_volumes, 
                                                 labels=concat_labels, title='using brain volumes')

Folds: 100%|██████████████████████████████████████| 5/5 [00:05<00:00,  1.06s/it]

Best Results using brain volumes:
Best Accuracy: 0.9400
Best Precision: 1.0000
Best Recall: 0.5000
Best F1 Score: 0.6700
Best Confusion Matrix:
[[29  0]
 [ 2  2]]
Best Parameters (C and Gamma): {'C': 100, 'gamma': 10}


In [143]:
# Test for independence of groups between rrms and pp/spms based on volumes


volumes_ms_rr = [normalized_v_ms[i] for i, value in enumerate(ms_type_labels) if value == 0]
volumes_ms_psp = [normalized_v_ms[i] for i, value in enumerate(ms_type_labels) if value == 1]

flat_list_rr = [item for sublist in volumes_ms_rr for item in sublist]
flat_list_psp = [item for sublist in volumes_ms_psp for item in sublist]

# Perform t-test
t_statistic_phe, p_value_phe = ttest_ind(np.array(flat_list_rr), np.array(flat_list_psp))

# Print the results
print(f'T-statistic: {t_statistic_phe}')
print(f'P-value: {p_value_phe}')

T-statistic: 1.7383999340396554
P-value: 0.08216792305578006


<a id='a5.6.2'></a>
<h3>Thalamic volumes</h3>

In [144]:
# we subset the values for left and right Thalamus

thalamic_volumes_ms = volumes_patients[['32', '39']]
thalamic_volumes_hv = volumes_controls[['32', '39']]
thalamic_volumes_ms

,32,39
0,7464,6720
1,9480,9208
2,8912,8936
3,8728,8600
4,8784,8552
...,...,...
142,8088,7456
143,6968,6392
144,8664,8696
145,7744,7064


In [145]:
th_ms = thalamic_volumes_ms.values[:,0:].tolist()
th_hv = thalamic_volumes_hv.values[:,0:].tolist()
thal = th_ms + th_hv

In [146]:
# we normalize values

flattened_list_t = [item for sublist in thal for item in sublist]

# Find the global min and max values
global_min_t = min(flattened_list_t)
global_max_t = max(flattened_list_t)

print(global_min_t)
print(global_max_t)

normalized_list_t = [(x - global_min_t) / (global_max_t - global_min_t) for x in flattened_list_t]

num_elements_t = len(th_ms[0])
normalized_volumes_t = [normalized_list_t[i:i + num_elements_t] for i in range(0, len(normalized_list_t), num_elements_t)]

print(max([max(i) for i in normalized_volumes_t]))
print(min([min(i) for i in normalized_volumes_t]))


376
10344
1.0
0.0


In [147]:
normalized_th_ms = normalized_volumes_t[:147]
normalized_th_hv = normalized_volumes_t[147:]
print(len(normalized_th_ms))
print(len(normalized_th_hv))

147
18


In [148]:
# we perform a t-test for independence between patients and controls 

flat_list1t = [item for sublist in normalized_th_ms for item in sublist]
flat_list2t = [item for sublist in normalized_th_hv for item in sublist]

# Perform t-test
t_statistic_t, p_value_t = ttest_ind(np.array(flat_list1t), np.array(flat_list2t))

# Print the results
print(f'T-statistic: {t_statistic_t}')
print(f'P-value: {p_value_t}')

T-statistic: -5.228102807167816
P-value: 3.0559398745953023e-07


In [149]:
# we perform a t-test for independence between phenotype groups

volumes_ms_rr_t = [normalized_th_ms[i] for i, value in enumerate(ms_type_labels) if value == 0]
volumes_ms_psp_t = [normalized_th_ms[i] for i, value in enumerate(ms_type_labels) if value == 1]

flat_list_rr_t = [item for sublist in volumes_ms_rr_t for item in sublist]
flat_list_psp_t = [item for sublist in volumes_ms_psp_t for item in sublist]

# Perform t-test
t_statistic_phe_t, p_value_phe_t = ttest_ind(np.array(flat_list_rr_t), np.array(flat_list_psp_t))

# Print the results
print(f'T-statistic: {t_statistic_phe_t}')
print(f'P-value: {p_value_phe_t}')

T-statistic: 1.6785661595692423
P-value: 0.09430642250575426


In [150]:
a_th, p_th, r_th, f1_th = classify_graph_skf(data=normalized_volumes_t, 
                                             labels=concat_labels, title='using only thalamic volumes')

Folds: 100%|██████████████████████████████████████| 5/5 [00:04<00:00,  1.17it/s]

Best Results using only thalamic volumes:
Best Accuracy: 0.6700
Best Precision: 0.1000
Best Recall: 0.3300
Best F1 Score: 0.1500
Best Confusion Matrix:
[[21  9]
 [ 2  1]]
Best Parameters (C and Gamma): {'C': 200, 'gamma': 0.01}


<a id='a5.6.3'></a>
<h3>Classification using the 5 nodes with the biggest difference in volumes between patients and controls</h3>

In [151]:
# We subtract volumes to identify the 5 nodes with the biggest difference between groups
array1 = np.array(normalized_v_ms)
array2 = np.array(normalized_v_hv)

mean_vol_ms = np.mean(array1, axis=0)
mean_vol_hv = np.mean(array2, axis=0)

vol_difference = list(np.subtract(mean_vol_hv, mean_vol_ms))

In [152]:
sorted_indices = np.argsort(vol_difference)[::-1]
top_5_indices = sorted_indices[:5]
top_5_nodes = [index + 1 for index in top_5_indices]
top_5_nodes

[71, 26, 22, 73, 67]

In [153]:
# We show the names of the identified nodes
corresponding_names = [(index, nodes_dict.get(index)) for index in top_5_nodes]
print(corresponding_names)

[(71, 'ctx-rh-superiorfrontal'), (26, 'ctx-lh-superiorfrontal'), (22, 'ctx-lh-precentral'), (73, 'ctx-rh-superiortemporal'), (67, 'ctx-rh-precentral')]


In [154]:
top_volumes_ms = volumes_patients[[str(v) for v in top_5_nodes]]
top_volumes_hv = volumes_controls[[str(v) for v in top_5_nodes]]
top_volumes_ms

,71,26,22,73,67
0,31088,26560,15312,16560,13696
1,34160,29336,16544,19168,16176
2,29896,24512,15504,18824,13696
3,33904,27808,17816,21024,16216
4,35600,30088,15584,21200,14696
...,...,...,...,...,...
142,34304,30856,15112,18112,13632
143,30408,26576,12800,16376,11784
144,39072,30384,16384,20936,16016
145,38984,33160,16496,20208,15312


In [155]:
top_volumes = [[sublist[i] for i in top_5_indices] for sublist in normalized_volumes]

In [156]:
normalized_top5_ms = top_volumes[:147]
normalized_top5_hv = top_volumes[147:]
print(len(normalized_top5_ms))
print(len(normalized_top5_hv))

147
18


In [157]:
# We perform a t-test for independence between groups using the top 5 most different volumes

flat_listtop5_1 = [item for sublist in normalized_top5_ms for item in sublist]
flat_listtop5_2 = [item for sublist in normalized_top5_hv for item in sublist]

# Perform t-test
t_statistic_top5, p_value_top5 = ttest_ind(np.array(flat_listtop5_1), np.array(flat_listtop5_2))

# Print the results
print(f'T-statistic: {t_statistic_top5}')
print(f'P-value: {p_value_top5}')

T-statistic: -2.5644489898671288
P-value: 0.0105103277181547


In [158]:
a_5, p_5, r_5, f1_5 = classify_graph_skf(data=top_volumes, 
                   labels=concat_labels, title='using the volumes of the 5 most different nodes')

Folds: 100%|██████████████████████████████████████| 5/5 [00:04<00:00,  1.14it/s]

Best Results using the volumes of the 5 most different nodes:
Best Accuracy: 0.7600
Best Precision: 0.1400
Best Recall: 0.3300
Best F1 Score: 0.2000
Best Confusion Matrix:
[[24  6]
 [ 2  1]]
Best Parameters (C and Gamma): {'C': 100, 'gamma': 10}


In [159]:
# we check for independence between phenotype groups

volumes_ms_rr_top5 = [normalized_top5_ms[i] for i, value in enumerate(ms_type_labels) if value == 0]
volumes_ms_psp_top5 = [normalized_top5_ms[i] for i, value in enumerate(ms_type_labels) if value == 1]

flat_list_rr_top5 = [item for sublist in volumes_ms_rr_top5 for item in sublist]
flat_list_psp_top5 = [item for sublist in volumes_ms_psp_top5 for item in sublist]

# Perform t-test
t_statistic_phe_top5, p_value_phe_top5 = ttest_ind(np.array(flat_list_rr_top5), np.array(flat_list_psp_top5))

# Print the results
print(f'T-statistic: {t_statistic_phe_top5}')
print(f'P-value: {p_value_phe_top5}')

T-statistic: 1.027696517874914
P-value: 0.3044316103444145


<a id='a5.6.4'></a>
<h3>Results</h3>

In [160]:
mix_graph_vol= pd.MultiIndex.from_product([['Brain volumes'], ['all', 'thalamic', 
                                                    '5 most different']])
table_vol = pd.DataFrame(columns=mix_graph_vol) 

table_vol['statistic'] = ['accuracy', 'precision', 'recall', 'F1']
table_vol[('Brain volumes', 'all')] = [a_vol, p_vol, r_vol, f1_vol]
table_vol[('Brain volumes', 'thalamic')] = [a_th, p_th, r_th, f1_th]
table_vol[('Brain volumes', '5 most different')] = [a_5, p_5, r_5, f1_5]
table_vol = table_vol.set_index('statistic')
table_vol.index.name=None

In [161]:
table_vol

Brain volumes                          
                    all thalamic 5 most different
accuracy           0.94     0.67             0.76
precision          1.00     0.10             0.14
recall             0.50     0.33             0.33
F1                 0.67     0.15             0.20

<a id='a5.7'></a>
<h2>Summary of classification results</h2>

In [162]:
summary_class = pd.concat([table_connectivity, table_graph, table_vol], axis=1)
summary_class

Connectivity                                   Graph  \
            structural functional combined structural strength   
accuracy           1.0       0.94     0.97                0.97   
precision          1.0       0.94     0.97                0.75   
recall             1.0       1.00     1.00                1.00   
F1                 1.0       0.97     0.98                0.86   

                                                       Brain volumes           \
          functional rich club vector of node measures           all thalamic   
accuracy                  0.94                    0.97          0.94     0.67   
precision                 0.67                    0.75          1.00     0.10   
recall                    0.67                    1.00          0.50     0.33   
F1                        0.67                    0.86          0.67     0.15   

                            
          5 most different  
accuracy              0.76  
precision             0.14  
recall                0.33  
F1                    0.20

<br>

<a id='a6'></a>
<h1>Comparison of strength for connectogram visualization</h1>

In these last cells we will export the the differences in strength between patients and controls for visualization purposes. 

In [163]:
# We subset the nodes where we obtained a significant difference in nodal strength

ms_strength_nodes = np.transpose(significant_nodes_ms_strength).tolist()
hv_strength_nodes = np.transpose(significant_nodes_hv_strength).tolist()

In [164]:
significant_nodes_ms_strength, significant_nodes_hv_strength, node_list = stats_nodal_test(ms_st, hv_st, verbose=False, sn=True)

Amount of significant nodes: 54/76
List of significant nodes: [2, 3, 4, 5, 6, 8, 9, 10, 11, 13, 14, 15, 17, 19, 20, 21, 22, 23, 25, 26, 27, 28, 29, 32, 33, 36, 38, 39, 40, 43, 44, 45, 47, 48, 51, 53, 54, 55, 56, 58, 59, 60, 62, 63, 64, 65, 66, 67, 68, 70, 71, 72, 73, 74]
Test used: mann-whitney


/home/vant/anaconda3/envs/svm/lib/python3.11/site-packages/scipy/stats/_morestats.py:1882: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [165]:
# We subtract differences between controls and patients, and we export the data as a file,
# containing the list of 54 nodes where we found significant differences, and their values 

ms_strength_array = np.array(ms_strength_nodes)
average_strenght_nodes_ms = np.mean(ms_strength_array, axis=0)

In [166]:
hv_strength_array = np.array(hv_strength_nodes)
average_strenght_nodes_hv = np.mean(hv_strength_array, axis=0)

In [167]:
strenght_substract = average_strenght_nodes_hv - average_strenght_nodes_ms
strength_difference = strenght_substract.tolist()
print(strength_difference)

[1.1738492929338413, 2.0646539783425943, 0.9399113853953045, 1.9256643787616294, 2.2205081946261487, 2.176947737388506, 2.5701400944308013, 1.7009351330681035, 2.4373601113541525, 1.9834546568647369, 1.2877271411019056, 1.7847869484231556, 1.1341160881280494, 2.116120919989134, 2.298275747910388, 1.3605665572515129, 1.2420825748303166, 2.637478776938579, 1.2710064404205035, 1.7795681745536314, 2.774998752293275, 1.8935526337900583, 2.259448633800531, 1.5886548802273026, 1.0514206089430616, 3.088031095461824, 1.3080498589655623, 1.5674358752353363, 2.080312756986679, 2.9421166878741083, 0.8671094625240068, 1.7062176656812387, 1.3307830921273514, 2.584403988948919, 2.3564939043933606, 2.054779239191717, 2.814316623286409, 1.6101526461938036, 2.3802871630316194, 2.2336891514567796, 1.4537105672049755, 1.4311224236764915, 1.5763673927580957, 1.1213451245930486, 2.3517597497661207, 1.5955297792617777, 0.9172149791030186, 1.260230785848421, 2.3135822965991792, 1.903825948560474, 1.9519656637

In [168]:
strength_dict = dict(zip(node_list, strength_difference))

In [169]:
import pickle

file_name = 'nodal_strengths.pkl'

with open(file_name, 'wb') as file:
    pickle.dump(strength_dict, file)

<a id='a7'></a>
<h1>Preliminary conclusions</h1>

Statistical tests have confirmed sgnificant differences between patients and controls when using connectivity matrices, graph measures and brain volume measures. We haven't been able to find significant differences between phenotype groups, and thus we have restricted our classification tasks to discerning between patients and controls. We have obtained the highest accuracy of classification when using structural FA connectivity, though functional and combined matrices have also yielded high accuracy (we combined functional and structural matrices through a simple weighted sum procedure, giving 75% importance to structural connectivity). Concerning global graph measures, we only obtain statistically significant differences between groups using network global strength of structural matrices. Remarkably, rich club measure of functional connectivity also yields significant difference between groups, though the precision it delivers in SVM classification tasks is sensibly lower compared to strength. On the other hand, nodal graph measures reveal significant differences on different amounts of nodes accross different types of measure. We used that fact to create a heterogeneous vector, formed by an ordered mixture of nodal graph measures derived from both structural and fucntional connectivity, to describe each subject, and we used these vectors in an SVM classification task, obtaining high levels of accuracy. Finally, we explored the capacity of brain volume data to discern between patents and controls, this investigating the role of brain atrophy in MS. We characterized subjects using three different aggregations of data: their complete brain volume information, the volumes of only thalamic nodes, and the 5 nodes where the volume difference is bigger, obtaining different results that we gather in our result summary table in section 6. 